Project Goal : To build a CNN-Classifier which can classify whether inputted image is a cat or a dog



In [3]:

import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import tensorflow
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
train_path = r"C:\Image Detection and Recognition\training_set\training_set"
test_path = r"C:\Image Detection and Recognition\test_set\test_set"

#Below variables are only for visualisation

train_dogs = r"training_set\dogs"
train_cats = r"training_set\cats"


Usage of ImageDataGenerator is done, which basically is a tool to convert Image Data into dimensional arrays, which has various parameters for image generation to prevent overfitting for CNNs.



Image Generator is an object, to get the actual data, flow_from_directory function is used, which is an iterator and will read the data in a usable form for the model.

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)
test_datagen = ImageDataGenerator(
    rescale = 1./255,
   
)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (150,150),
    batch_size = 32,
    class_mode = 'binary'
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size = (150,150),
    batch_size = 32,
    class_mode = 'binary'
)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

The model consists of:

1) 32-neuron Convolution Layer
2) 64-neuron Convolution Layer
3) 128-neuron Convolution Layer
4) 512-neuron Dense Layer
5) Output Layer

The first 4 layers have sub-layers which are required to tackle with overfitting,which are Max Pooling Layers.

Overfitting prevention measures such as Batch Normalisation and Dropout are used in every main layer.

In [5]:
model = Sequential(
    [
        # LAYER 1

        Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
        MaxPooling2D((2,2), padding='same'),
        BatchNormalization(),
        Dropout(0.3),

        # LAYER 2

        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D((2,2), padding='same'),
        BatchNormalization(),
        Dropout(0.3),

        # LAYER 3

        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D((2,2), padding='same'),
        BatchNormalization(),
        Dropout(0.4),

        # FINAL LAYER : Dense Layer + Output Layer
        
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')

    ]
)

model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics = ['accuracy'])
model.build(input_shape=(150,150,3))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 74, 74, 32)       128       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 74, 74, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0

In [6]:
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples//train_generator.batch_size,
    epochs = 50,
    validation_data = test_generator,
    validation_steps = test_generator.samples//test_generator.batch_size
)
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy : {test_acc:.2f}")

Epoch 1/50
250/250 [==============================] - 52s 175ms/step - loss: 3.6905 - accuracy: 0.5991 - val_loss: 1.1388 - val_accuracy: 0.5704
Epoch 2/50
250/250 [==============================] - 39s 157ms/step - loss: 0.6903 - accuracy: 0.6808 - val_loss: 0.6485 - val_accuracy: 0.6925
Epoch 3/50
250/250 [==============================] - 41s 162ms/step - loss: 0.5698 - accuracy: 0.7167 - val_loss: 0.5538 - val_accuracy: 0.7475
Epoch 4/50
250/250 [==============================] - 41s 164ms/step - loss: 0.5334 - accuracy: 0.7321 - val_loss: 0.6306 - val_accuracy: 0.7391
Epoch 5/50
250/250 [==============================] - 43s 173ms/step - loss: 0.5236 - accuracy: 0.7421 - val_loss: 0.8758 - val_accuracy: 0.7282
Epoch 6/50
250/250 [==============================] - 42s 167ms/step - loss: 0.4993 - accuracy: 0.7538 - val_loss: 0.4989 - val_accuracy: 0.7659
Epoch 7/50
250/250 [==============================] - 47s 186ms/step - loss: 0.4937 - accuracy: 0.7603 - val_loss: 0.5410 - val_ac

In [ ]:
x_batch, y_batch = next(train_generator)
plt.figure(figsize=(10,10))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(x_batch[i])
    plt.title(f"Label : {int(y_batch[i])}")
    plt.axis('off')
plt.show()

y_batch


Code below is for visualising how the model's accuracy is when its measured on the training data vs when its measured on the testing data. When the gap between training accuracy and validation (testing) accuracy is significant, its a sign of overfitting. In this project, the model overfits the data, however, it still has a validation accuracy of about 85%. It surely can be optimised by processes like data augmentation, but this project is intended to give myself an insight of how convolutional neural networks work.

In [ ]:
plt.plot(history.history['accuracy'], color = 'blue', label='train_accuracy')
plt.plot(history.history['val_accuracy'], color = 'red', label='val_accuracy')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

Code below is for saving the model, and using it directly for predictions : Main purpose being that it would be useful to load up the model and make a web UI for practice.


In [28]:
model.save('cnn_model.h5')